### Sentiment classification from movie reviews (IMBD dataset).

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

Let's do sentiment classification from movie reviews (IMBD dataset).

Data: 25000 movie reviews from IMDB, labelled good (1) or bad (0).

Strategy: Embed sentences in reviews, them learn structure of reviews with LSTM

In [3]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.datasets import imdb

In [4]:
max_features = 20000 # select only the 20000 most common items (words) from our vocabulary
maxlen = 80 # only allow sequence of length 80

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(x_train.shape, x_test.shape)

/home/hugo/local/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


(25000,) (25000,)


/home/hugo/local/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [5]:
# sequences shorter than maxlen items are padded with zeros at the end
# sequences longer than maxlen are cut
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [6]:
# define model
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid')) # output probability (between 0 and 1) of reviews being good or bad 

In [7]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [8]:
# train model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 99s 127ms/step - loss: 0.6934 - accuracy: 0.4929 - val_loss: 0.6932 - val_accuracy: 0.4972
Epoch 2/10
782/782 [==============================] - 112s 144ms/step - loss: 0.6931 - accuracy: 0.5054 - val_loss: 0.6930 - val_accuracy: 0.4999
Epoch 3/10
782/782 [==============================] - 101s 129ms/step - loss: 0.6929 - accuracy: 0.5095 - val_loss: 0.6927 - val_accuracy: 0.5367
Epoch 4/10
782/782 [==============================] - 107s 137ms/step - loss: 0.6927 - accuracy: 0.5210 - val_loss: 0.6925 - val_accuracy: 0.5420
Epoch 5/10
782/782 [==============================] - 108s 138ms/step - loss: 0.6925 - accuracy: 0.5294 - val_loss: 0.6923 - val_accuracy: 0.5279
Epoch 6/10
782/782 [==============================] - 109s 140ms/step - loss: 0.6921 - accuracy: 0.5232 - val_loss: 0.6921 - val_accuracy: 0.5109
Epoch 7/10
782/782 [==============================] - 113s 144ms/step - loss: 0.6919 - accuracy: 0.5477 - val_loss: 0.6918 - 

In [9]:
score = model.evaluate(x_test, y_test)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

782/782 [==============================] - 10s 13ms/step - loss: 0.6907 - accuracy: 0.5416
Test loss:  0.690742015838623
Test accuracy:  0.5415599942207336


In [10]:
# visualize model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
# save model as json and weights as hdf5
json_string = model.to_json() # alternative: model.to_yaml()
model.save_weights("model_weights_sentiment_imdb.h5")

# save full model (architecture + weights)
model.save("model_sentiment_imdb.h5")

In [14]:
from tensorflow.keras.models import model_from_json, load_model

# load model from json and set weights
model_json = model_from_json(json_string)
model_json.load_weights("model_weights_sentiment_imdb.h5")

# load full model
model_full = load_model("model_sentiment_imdb.h5")


In [15]:
model_json.summary()
model_full.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
__________________________